In [1]:
import ee
from pathlib import Path
import pandas as pd
import ast

ee.Authenticate() #Uncomment this whenever needed, once done usually not needed for 1-2 days
ee.Initialize(project='raman-461708')

AEZ_no = 1
# List all assets in the folder
asset_list = ee.data.listAssets({'parent': 'projects/raman-461708/assets/'})['assets']

# Filter for boundaries assets that match the AEZ_no
AEZ_no = 1  # or any AEZ_no you want
boundary_assets = [
    asset['name'] for asset in asset_list
    if f'AEZ_{AEZ_no}_boundaries' in asset['name']
]

# Load and merge all matching boundaries for this AEZ_no
aez_boundaries = ee.FeatureCollection([])
for asset_path in boundary_assets:
    fc = ee.FeatureCollection(asset_path)
    aez_boundaries = aez_boundaries.merge(fc)

import pandas as pd

# Read the CSV file
df = pd.read_csv("AEZ_"+str(AEZ_no)+"/status.csv")

# Function to create a tile (rectangle polygon) from diagonal points
def create_tile_from_points(points):
    # points is a string representation of a tuple, e.g. "((lat1, lon1), (lat2, lon2))"
    if isinstance(points, str):
        points_tuple = ast.literal_eval(points)
    else:
        points_tuple = points
    (lon1, lat1), (lon2, lat2) = points_tuple
    # Get min/max for lat/lon to define the rectangle
    min_lat, max_lat = min(lat1, lat2), max(lat1, lat2)
    min_lon, max_lon = min(lon1, lon2), max(lon1, lon2)
    # Return the four corners in order (clockwise or counterclockwise)
    return [
        (min_lat, min_lon),
        (min_lat, max_lon),
        (max_lat, max_lon),
        (max_lat, min_lon),
        (min_lat, min_lon)  # close the polygon
    ]

# Create a new column 'tile' with the rectangle polygon for each row
df['tile'] = df['points'].apply(create_tile_from_points)
tiles = df['tile'].tolist()

samples = []
import concurrent.futures
import time

# Function to chunk a list into sublists of size p
#def chunk_list(lst, p):
#    return [lst[i:i + p] for i in range(0, len(lst), p)]

# Define the number of tiles per group
#p = 1

# Chunk the tiles into groups
#tile_groups = chunk_list(tiles, p)

def fc_to_df(fc):
    # Get the data from the FeatureCollection
    features = fc.getInfo()['features']
    
    # Extract properties and geometry
    data = [feature['properties'] for feature in features]
    
    return pd.DataFrame(data)

def process_tile(tile, index):
    tile = ee.Geometry.Polygon(tile).buffer(-1)
    
    #aez_region = ee.FeatureCollection("users/mtpictd/agro_eco_regions").filter(ee.Filter.eq("ae_regcode", 7)).filter(tile)
    #g_embs = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL").filterDate('2024-01-01', '2025-01-01').filterBounds(tile).mosaic()

    mapping = {
        "farm": 1,
        "plantation": 2,
        "scrubland": 3,
        "rest": 0
    }
    reversed_mapping = {v: k for k, v in mapping.items()}
    reversed_ee_mapping = ee.Dictionary(reversed_mapping)

    easy_farm = [
        ee.Filter.gte("rect", 0.67),
        ee.Filter.gt("size", 500),
        ee.Filter.lt("size", 2000),
        ee.Filter.lt("ent", 1)
        ]
    easy_scrubland = [
        ee.Filter.gte("size", 60000),
        ee.Filter.lte("size", 5000000)
        ]
    easy_plantation = [
        ee.Filter.lt("area", 20000),
        ee.Filter.gt("area", 1000)
    ]

    all = aez_boundaries.filterBounds(tile)
    farm = all.filter(ee.Filter.And(*easy_farm))
    scrubland = all.filter(ee.Filter.And(easy_scrubland))
    plantation = all.filter(ee.Filter.eq("class", "plantation")).map(lambda x: x.set("area", x.geometry().area())).filter(ee.Filter.And(easy_plantation))
    mapping = {
        "farm": 1,
        "plantation": 2,
        "scrubland": 3,
        "rest": 0
    }
    reversed_mapping = {v: k for k, v in mapping.items()}
    reversed_ee_mapping = ee.Dictionary(reversed_mapping)

    easy_farm = [
        ee.Filter.gte("rect", 0.67),
        ee.Filter.gt("size", 500),
        ee.Filter.lt("size", 2000),
        ee.Filter.lt("ent", 1)
        ]

    easy_scrubland = [
        ee.Filter.gte("size", 60000)
        ]

    
    
    #Filter out farms which doesnot have 3 nearby farms (Removing solo farms inside scrublands)
    farm_buffer = farm.map(lambda x: x.buffer(10))
    # Merge touching farms into clusters
    farm_buffer_union = farm_buffer.union(maxError=1)
    geoms = farm_buffer_union.geometry().dissolve().geometries()
    geoms = geoms.map(lambda geom: ee.Feature(ee.Geometry(geom)))
    # Flatten clusters into individual polygons
    clusters = ee.FeatureCollection(geoms)
    clusters = clusters.map(lambda x: x.set("count", farm.filterBounds(x.geometry()).size())).filter(ee.Filter.lt('count', 3))
    farm = farm.filterBounds(clusters)
    
    #Filtering out scrublands
    lulc_v3 = ee.Image("projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2023_2024").clip(tile)
    classes_of_interest = [8, 9, 10, 11]
    
    
    # Build a binary mask for classes of interest (1 = of interest, 0 = otherwise)
    interest_mask = (
        lulc_v3
        .remap(classes_of_interest, ee.List.repeat(1, len(classes_of_interest)), 0)
        .rename('interestMask')
        .unmask(0)
    )

    # Pixel area bands
    px_area       = ee.Image.pixelArea().rename('total_area')
    interest_area = px_area.updateMask(interest_mask).rename('interest_area')

    # Stack and clip to the tile
    area_stack = interest_area.addBands(px_area).clip(tile)

    # Sum both bands over all polygons in one call
    summed = area_stack.reduceRegions(
    collection=scrubland,
    reducer=ee.Reducer.sum(),
    scale=10
    )

    # Compute percentage per feature (no if-condition needed)
    with_percent = summed.map(
        lambda f: f.set(
            'percent_interest',
            ee.Number(f.get('interest_area'))
            .divide(ee.Number(f.get('total_area')))
            .multiply(100)
        )
    )

    # Filter: < 50% interest AND strictly inside the ROI
    scrubland = (
        with_percent
        .filter(ee.Filter.lt('percent_interest', 50))
    )
    # Desired samples per class
    class_values = [(0,0),(1, 150), (2, 150), (3, 150)]
    class_band = 'label'

    # Build the label raster (priority = last paint wins for overlaps).
    # If you need explicit precedence, keep this order intentional.
    label_image = (
        ee.Image(0).byte().rename(class_band)
        .paint(farm,       mapping["farm"])
        .paint(scrubland,  mapping["scrubland"])
        .paint(plantation, mapping["plantation"])
        .clip(tile)
    )

    # Prepare lists for stratified sampling
    class_values_list  = [v for v, _ in class_values]
    class_points_list  = [n for _, n in class_values]

    # One stratified sample call; returns up to the requested points per class
    all_samples = label_image.stratifiedSample(
        numPoints=sum(class_points_list),
        classBand=class_band,
        classValues=class_values_list,
        classPoints=class_points_list,
        scale=10,                 # your LULC pixel size
        region=tile,
        seed=42,
        dropNulls=True,
        geometries=True
    )
    
    """
    label_image = ee.Image(0).rename("label")
    farm_mask = label_image.clip(farm).mask()
    scrubland_mask = label_image.clip(scrubland).mask()
    plantation_mask = label_image.clip(plantation).mask()

    label_image = label_image.where(farm_mask, mapping["farm"]).where(scrubland_mask, mapping["scrubland"]).where(plantation_mask, mapping["plantation"])
    
    # Classes to sample (exclude background = 0)
    class_values = [(1, 150), (2, 150), (3, 150)]

    # Create masks for all classes and combine them into a single mask
    class_band = 'label'
    class_values_list = [class_val for class_val, _ in class_values]
    class_points_list = [points for _, points in class_values]

    # Sample uniformly from the masked image using a single call to stratifiedSample
    all_samples = label_image.stratifiedSample(
        numPoints=sum(class_points_list),  # Total number of points needed
        classBand=class_band,
        classValues=class_values_list,
        classPoints=class_points_list,
        scale=10,  # Adjust the scale as needed
        region=tile.geometry(),
        seed=42,
        dropNulls=True,
        geometries=True
    )

    # Filter samples to ensure each class has the correct number of points
    def filter_samples(samples, class_val, points):
        return samples.filter(ee.Filter.eq(class_band, class_val)).limit(points)

    filtered_samples_list = [
        filter_samples(all_samples, class_val, points)
        for class_val, points in class_values
    ]

    # Merge all filtered samples into a single FeatureCollection
    all_filtered_samples = ee.FeatureCollection(filtered_samples_list).flatten()
    return all_filtered_samples
    """
    task = ee.batch.Export.table.toDrive(
        collection=all_samples,
        description=str(AEZ_no) + "_tile_" + str(index),
        fileFormat='CSV',
        folder='Scrubland_Field_Delineation/AEZ_' + str(AEZ_no),
        fileNamePrefix='tile_' + str(index),
    )
    task.start()
    print(f"tile_" + str(index) + " started")
    while True:
        status = task.status()['state']
        print(f"Task tile_{index} status: {status}")
        if status in ['COMPLETED', 'FAILED', 'CANCELLED']:
            break
        time.sleep(30) 
    #all_filtered_samples_df = fc_to_df(all_filtered_samples)
    #all_filtered_samples_df.to_csv('output_'+str(index)+'.csv', index=False)
    #print("Done tile no ", index)
    

#for i in range(29, len(tiles)):
#    process_tile(tiles[i], i)

In [4]:
scrubland = process_tile(tiles[27],27)

tile_27 started
Task tile_27 status: READY
Task tile_27 status: RUNNING
Task tile_27 status: COMPLETED


In [18]:
tiles[27]

[(78.11004638671875, 35.01875037943838),
 (78.11004638671875, 35.05473209995077),
 (78.15399169921875, 35.05473209995077),
 (78.15399169921875, 35.01875037943838),
 (78.11004638671875, 35.01875037943838)]

In [ ]:
import geemap
import ee
import pandas as pd
import ast

# Set AEZ number here
AEZ_no = 4

# Initialize Earth Engine
# ee.Authenticate()
# ee.Initialize()

# Read the CSV
df = pd.read_csv(f"AEZ_{AEZ_no}/status.csv")

# Function to create a rectangle polygon from the 'points' string
def create_polygon(points):
    (lat1, lon1), (lat2, lon2) = ast.literal_eval(points)
    min_lat, max_lat = min(lat1, lat2), max(lat1, lat2)
    min_lon, max_lon = min(lon1, lon2), max(lon1, lon2)
    coords = [
        [min_lon, min_lat],
        [min_lon, max_lat],
        [max_lon, max_lat],
        [max_lon, min_lat],
        [min_lon, min_lat]
    ]
    return ee.Geometry.Polygon([coords])

# Build list of ee.Feature
features = []
for _, row in df.iterrows():
    geom = create_polygon(row['points'])
    props = row.drop('points').to_dict()
    features.append(ee.Feature(geom, props))

# Create the FeatureCollection
boundaries_boxes = ee.FeatureCollection(features)


# Example for boundaries if needed:
# boundaries_0 = ee.FeatureCollection(f"projects/raman-461708/assets/AEZ_{AEZ_no}_boundaries_0")
# boundaries_1 = ee.FeatureCollection(f"projects/raman-461708/assets/AEZ_{AEZ_no}_boundaries_1")
# boundaries = boundaries_0.merge(boundaries_1)

aez = ee.FeatureCollection("users/mtpictd/agro_eco_regions").filter(ee.Filter.eq("ae_regcode", AEZ_no))
tiles = ee.FeatureCollection(f"projects/raman-461708/assets/AEZ_{AEZ_no}_tiles")

# Example for centering the map (if you have tiles)
# bounds = tiles.geometry().bounds()
# coords = bounds.centroid(10).coordinates().getInfo()
# center = [coords[1], coords[0]]

# If you don't have tiles, use a default center
center = [21.54, 74.51]  # Replace with your region of interest

# Create a geemap interactive map
Map = geemap.Map(center=center, zoom=7, basemap='SATELLITE')
Map.addLayer(tiles, {}, "Tiles")

Map.addLayer(boundaries_boxes, {}, "Boundaries")
Map.addLayer(aez, {}, "AEZ")

Map

Map(center=[21.54, 74.51], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright'…

In [ ]:
tiles.size()

In [2]:
# pip install geemap earthengine-api
import geemap


# --- Inputs ---
# 'aez' should be an ee.FeatureCollection with a numeric/string property 'ae_regcode'
# Example (replace with your asset if needed):
# aez = ee.FeatureCollection('users/your_username/aez')

label_prop = 'ae_regcode'  # change if your field name differs

aez = ee.FeatureCollection("users/mtpictd/agro_eco_regions")
# pip install geemap earthengine-api


# --- Inputs ---
# 'aez' should be an ee.FeatureCollection with a property 'ae_regcode'
# Example:
# aez = ee.FeatureCollection('users/your_username/aez')

label_prop = 'ae_regcode'  # change if your label field differs

# Style boundaries (transparent fill)
styled = aez.style(
    color='#1f77b4',
    width=2,
    fillColor='00000000'
)

def interior_label_point(f):
    geom = f.geometry()
    # 1) Try centroid
    centroid = ee.Geometry(geom.centroid(maxError=1))
    # 2) If centroid not inside, fall back to one random point inside polygon
    pt = ee.Algorithms.If(
        geom.contains(centroid, 1),
        centroid,
        ee.FeatureCollection.randomPoints(region=geom, points=1, maxError=1).first().geometry()
    )
    # Build label text (works for numeric or string)
    label = ee.String(f.get(label_prop))
    return ee.Feature(ee.Geometry(pt), {'label': label})

label_fc = aez.map(interior_label_point)

# Map
m = geemap.Map()
m.addLayer(styled, {}, 'AEZ boundaries')

# Add labels (requires geemap >= 0.27.0)
m.add_labels(
    label_fc,
    'label',                 # column (positional)
    font_size='14pt',        # <-- must include 'pt'
    font_color='black',
    outline_color='white',
    outline_width='3pt',     # safer to give 'pt' here too
    name='AEZ labels'
)

m.centerObject(aez, 5)
m


Map(center=[23.22104337560591, 79.46098827567988], controls=(WidgetControl(options=['position', 'transparent_b…

In [5]:
import geemap
import ee
import pandas as pd
import ast

ee.Initialize(project='raman-461708')

all_tiles = ee.FeatureCollection([])
all_boundaries = ee.FeatureCollection([])
all_aez = ee.FeatureCollection([])

for AEZ_no in range(1, 21):
    # Try to read the CSV for this AEZ (skip if not found)
    try:
        df = pd.read_csv(f"AEZ_{AEZ_no}/status.csv")
    except FileNotFoundError:
        print(f"AEZ_{AEZ_no}/status.csv not found, skipping.")
        continue

    # Build boundaries_boxes from CSV
    def create_polygon(points):
        (lat1, lon1), (lat2, lon2) = ast.literal_eval(points)
        min_lat, max_lat = min(lat1, lat2), max(lat1, lat2)
        min_lon, max_lon = min(lon1, lon2), max(lon1, lon2)
        coords = [
            [min_lon, min_lat],
            [min_lon, max_lat],
            [max_lon, max_lat],
            [max_lon, min_lat],
            [min_lon, min_lat]
        ]
        return ee.Geometry.Polygon([coords])

    features = []
    for _, row in df.iterrows():
        geom = create_polygon(row['points'])
        props = row.drop('points').to_dict()
        features.append(ee.Feature(geom, props))
    boundaries_boxes = ee.FeatureCollection(features)
    all_boundaries = all_boundaries.merge(boundaries_boxes)

    # AEZ boundary and tiles
    aez = ee.FeatureCollection("users/mtpictd/agro_eco_regions").filter(ee.Filter.eq("ae_regcode", AEZ_no))
    all_aez = all_aez.merge(aez)
    try:
        tiles = ee.FeatureCollection(f"projects/raman-461708/assets/AEZ_{AEZ_no}_tiles")
        all_tiles = all_tiles.merge(tiles)
    except Exception as e:
        print(f"Tiles for AEZ_{AEZ_no} not found or error: {e}")

Map = geemap.Map(center=[21.54, 74.51], zoom=5, basemap='SATELLITE')
Map.addLayer(all_tiles, {}, "All Tiles")
Map.addLayer(all_boundaries, {}, "All Boundaries")
Map.addLayer(all_aez, {}, "All AEZs")
Map

AEZ_20/status.csv not found, skipping.


Map(center=[21.54, 74.51], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright'…

In [8]:
import geemap

AEZ_no = 17
roi = ee.FeatureCollection("users/mtpictd/agro_eco_regions").filter(ee.Filter.eq("ae_regcode", AEZ_no)) 
original_lulc_img = ee.Image('projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2023_2024').clip(roi).select('predicted_label')
final_lulc_img = ee.Image('projects/raman-461708/assets/AEZ_'+str(AEZ_no)+'_v4')
m = geemap.Map()
#m.centerObject(farm.limit(1))


url = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
m.layout.height = '1000px'
m.add_tile_layer(url, name="Google Map", attribution="Google")
#m.add_basemap(, )
m.addLayerControl()
#m.addLayer(fields_boundary, {}, 'Fields')

palette = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e'
]

palette_corrected = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e', '29554E'
]

vis_params = {
    'min': 0,
    'max': 12,
    'palette': palette
}

vis_params_corrected = {
    'min': 0,
    'max': 13,
    'palette': palette_corrected
}

labels = """0. Background
1. Built-up
2. Water (Kharif)
3. Water (Kharif + Rabi)
4. Water (Kharif + Rabi + Zaid)
5. Croplands
6. Tree/Forest
7. Barren Land
8. Single Kharif (Light blue)
9. Single Non-Kharif
10. Double (Dark-Blue)
11. Triple
12. Shrub_Scrub
13. Plantation"""
labels = [i.split(". ")[-1] for i in labels.split("\n")]

palette_ = ['3bef34', '991695', '0000FF']
vis_params_ = {
    'min': 0,
    'max': 2,
    'palette': palette_
}

m.addLayer(final_lulc_img.select("predicted_label"), vis_params_corrected, 'LULC_corrected_4')
m.addLayer(original_lulc_img, vis_params, 'LULC_original')
m.addLayer(all_boundaries.filterBounds(roi), {}, "Sampling regions")

m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…